<a href="https://colab.research.google.com/github/felixzhao/stat501-project/blob/main/notebooks/3_1_FZ_process_texts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:

from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Study@NZ/VUW@NZ/STATS 501/Project"

Mounted at /content/drive
 CoachHandbook.docx	'Data Dictionary.gdoc'		   Texts.csv
 csv			 pre_process_round_1
'Data Dictionary.docx'	'Stats Consulting Data (1).xlsx'


# load df

In [3]:
texts_path = '/content/drive/My Drive/Study@NZ/VUW@NZ/STATS 501/Project/Texts.csv'

In [41]:
output_path = '/content/drive/My Drive/Study@NZ/VUW@NZ/STATS 501/Project/pre_process_round_1/citi_agg_texts.csv'

In [13]:
df = pd.read_csv(texts_path, low_memory=False, index_col=False)

In [17]:
if 'Unnamed: 0' in df.columns:
    df = df.drop('Unnamed: 0', axis=1)

## info

In [19]:
print(len(df.columns))
df.columns

13


Index(['partnerid', 'citizenid', 'Unnamed: 3', 'startdate', 'senderrole',
       'intervention', 'autogenerated', 'messagesentdatetime',
       'messageseendatetime', 'withvideo', 'attachmenttype',
       'videodurationseconds', 'messagetext_length'],
      dtype='object')

In [26]:
print(df.apply(lambda col: col.unique()))

partnerid                                                  [12003.0, nan]
citizenid               [810359, 811969, 816753, 816799, 816914, 82195...
Unnamed: 3              [803298, 811969, 816799, 831841, 845710, 83151...
startdate               [10/10/20, 28/10/20, 7/11/20, 14/11/20, nan, 2...
senderrole                                        [ADVISOR, CITIZEN, nan]
intervention                                           [False, True, nan]
autogenerated                                          [True, False, nan]
messagesentdatetime     [18:53.4, 59:46.9, 36:18.5, 50:47.4, 16:52.0, ...
messageseendatetime     [nan, 34:36.1, 11:32.9, 16:52.0, 06:21.1, 35:5...
withvideo                                              [False, True, nan]
attachmenttype                             [nan, VIDEO, LINK, PDF, IMAGE]
videodurationseconds    [nan, 150.0, 139.0, 84.0, 110.0, 108.0, 81.0, ...
messagetext_length      [90, 85, 578, 260, 54, 550, 92, 93, 845, 559, ...
dtype: object


In [22]:
df.describe(include='all')

,partnerid,citizenid,Unnamed: 3,startdate,senderrole,intervention,autogenerated,messagesentdatetime,messageseendatetime,withvideo,attachmenttype,videodurationseconds,messagetext_length
count,71248.0,1.048575e+06,1.048575e+06,71164,71242,71242,71242,71242,57318,71242,62394,17728.000000,1.048575e+06
unique,NaN,NaN,NaN,190,2,2,2,15056,11220,2,4,NaN,NaN
top,NaN,NaN,NaN,26/08/21,ADVISOR,True,False,01:00.0,46:48.1,False,PDF,NaN,NaN
freq,NaN,NaN,NaN,1413,63933,44580,56488,9505,120,51850,29494,NaN,NaN
mean,12003.0,6.602817e+04,6.140370e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137.613211,3.361396e+01
std,0.0,2.450908e+05,2.288775e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.965556,1.796612e+02
min,12003.0,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,0.000000e+00
25%,12003.0,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.000000,0.000000e+00
50%,12003.0,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113.000000,0.000000e+00
75%,12003.0,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,179.250000,0.000000e+00


# analysis one citizen

In [29]:
df.head()

,partnerid,citizenid,Unnamed: 3,startdate,senderrole,intervention,autogenerated,messagesentdatetime,messageseendatetime,withvideo,attachmenttype,videodurationseconds,messagetext_length
0,12003.0,810359,803298,10/10/20,ADVISOR,False,True,18:53.4,NaN,False,NaN,NaN,90
1,12003.0,811969,811969,10/10/20,CITIZEN,False,True,59:46.9,34:36.1,False,NaN,NaN,85
2,12003.0,811969,803298,10/10/20,ADVISOR,True,False,36:18.5,34:36.1,True,VIDEO,150.0,578
3,12003.0,811969,803298,10/10/20,ADVISOR,True,False,36:18.5,34:36.1,False,VIDEO,150.0,578
4,12003.0,811969,803298,10/10/20,ADVISOR,False,False,50:47.4,11:32.9,False,NaN,NaN,260


In [44]:
citi = df[df['citizenid'] == 1022212]
len(citi)

1

In [45]:
citi.head(2)

,partnerid,citizenid,Unnamed: 3,startdate,senderrole,intervention,autogenerated,messagesentdatetime,messageseendatetime,withvideo,attachmenttype,videodurationseconds,messagetext_length
22621,12003.0,1022212,1022212,NaN,CITIZEN,False,True,34:44.4,NaN,False,NaN,NaN,95


# agg data

- num_intervention
- n_autogenerated
- n_withvideo
- min_msg_len
- avg_msg_len
- max_msg_len

In [52]:
# Group by 'citizenid' and aggregate
agg_funcs = {
    'intervention': lambda x: (x == True).sum(),  # count of intervention when True
    'autogenerated': lambda x: (x == True).sum(),  # count of autogenerated when True
    'withvideo': lambda x: (x == True).sum(),  # count of withvideo when True
    'messagetext_length': ['min', 'mean', 'max']  # min, avg, and max of messagetext_length

}

citi_agg_df = df.groupby('citizenid').agg(agg_funcs).reset_index()

# Rename columns for better readability
citi_agg_df.columns = ['citizenid',
                  'count_intervention', 'count_autogenerated', 'count_withvideo',
                  'min_messagetext_length', 'avg_messagetext_length', 'max_messagetext_length']

citi_agg_df = citi_agg_df[citi_agg_df['citizenid'] != 0]

print(len(citi_agg_df))
citi_agg_df.head()

367


,citizenid,count_intervention,count_autogenerated,count_withvideo,min_messagetext_length,avg_messagetext_length,max_messagetext_length
1,810359,16,1,4,90,473.238095,591
2,811969,74,17,28,0,513.330357,1474
3,816592,31,7,31,90,647.911765,1313
4,816753,40,7,19,90,569.617284,1108
5,816799,47,14,37,0,497.467066,1345


# save citi agg to file

In [54]:
print(output_path)
citi_agg_df.to_csv(output_path, index=False)

/content/drive/My Drive/Study@NZ/VUW@NZ/STATS 501/Project/pre_process_round_1/citi_agg_texts.csv
